In [1]:
import numpy as np
import pandas as pd

In [2]:
X = np.load('/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/data/X.npy',allow_pickle= True)
y = np.load('/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/data/y.npy',allow_pickle= True)

In [3]:
X.shape

(41157,)

In [4]:
from Modules.gensim_vectorizers import FastText_vectorize
vec = FastText_vectorize("/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/fasttext/wiki-news-300d-1M.vec")

In [5]:
vec._build_vector()

999995it [01:25, 11707.51it/s]


In [6]:
vec._get_vector_list(X[0]).shape

(12, 300)

In [7]:
X, y = X[:2000] , y[:2000]

In [8]:
print(X.shape, y.shape)

(2000,) (2000,)


In [9]:
pd.DataFrame(y).value_counts()

2    825
0    819
1    356
dtype: int64

In [11]:
vec_list = [vec.padding_truncate(vec._get_vector_list(X[i]),max_length=20) for i in range(X.shape[0])]
# vec_list = vec.padding_truncate_lists(X,max_length=20)

In [15]:
vec_list = np.array(vec_list)

In [14]:
Vec_list.shape

(2000, 20, 300)

In [16]:
(SequenceLegnth , EmbeddingLength) = vec_list.shape[1:]

In [17]:
print(f"SequenceLegnth = {SequenceLegnth}, EmbeddingLength = {EmbeddingLength}")

SequenceLegnth = 20, EmbeddingLength = 300


In [18]:
np.save('/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/data/X_vec.npy',Vec_list)

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM , Bidirectional , Dropout
#to_categorical
from tensorflow.keras.utils import to_categorical

2022-07-26 21:09:18.360518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-26 21:09:18.360538: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [20]:
y = to_categorical(y)

In [21]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128, return_sequences=True), input_shape=(SequenceLegnth, EmbeddingLength)))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
model.summary()

2022-07-26 21:09:24.658844: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 21:09:24.659060: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-26 21:09:24.659333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-26 21:09:24.659615: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-26 21:09:24.659770: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 20, 256)          439296    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 20, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
Total params: 1,227,776
Trainable params: 1,227,776
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.add(Dense(units=3, activation='softmax'))

In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(vec_list, y, epochs=10, batch_size=32,validation_split=0.2)

Epoch 1/10


2022-07-26 21:09:35.313539: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 38400000 exceeds 10% of free system memory.


50/50 [==============================] - 13s 154ms/step - loss: 0.9189 - accuracy: 0.5713 - val_loss: 0.8341 - val_accuracy: 0.6075
Epoch 2/10
50/50 [==============================] - 6s 128ms/step - loss: 0.7575 - accuracy: 0.6906 - val_loss: 0.8228 - val_accuracy: 0.6625
Epoch 3/10
50/50 [==============================] - 6s 121ms/step - loss: 0.6709 - accuracy: 0.7244 - val_loss: 0.7811 - val_accuracy: 0.6625
Epoch 4/10
50/50 [==============================] - 6s 115ms/step - loss: 0.5968 - accuracy: 0.7619 - val_loss: 0.8339 - val_accuracy: 0.6425
Epoch 5/10
50/50 [==============================] - 6s 116ms/step - loss: 0.5466 - accuracy: 0.7850 - val_loss: 0.8953 - val_accuracy: 0.6275
Epoch 6/10
50/50 [==============================] - 5s 99ms/step - loss: 0.4688 - accuracy: 0.8281 - val_loss: 0.9034 - val_accuracy: 0.6525
Epoch 7/10
50/50 [==============================] - 5s 102ms/step - loss: 0.4294 - accuracy: 0.8394 - val_loss: 1.0471 - val_accuracy: 0.6375
Epoch 8/10
50/50 

In [25]:
model.evaluate(Vec_list, y)

 3/63 [>.............................] - ETA: 1s - loss: 0.2029 - accuracy: 0.9479

2022-07-26 21:10:37.553856: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 48000000 exceeds 10% of free system memory.


63/63 [==============================] - 2s 29ms/step - loss: 0.3784 - accuracy: 0.8860


[0.378403902053833, 0.8859999775886536]

In [26]:
from Modules.text import Preprocessing
prpcs = Preprocessing(lemmatize=True)

In [55]:
tweet  = ["I am a good person in this society and life for me is beautiful"]
tweet = prpcs.text_normalize(tweet[0])
# type(tweet[0])

In [56]:
tweet = prpcs.text_tokenize(tweet)

In [57]:
tweet

['good', 'person', 'society', 'life', 'beautiful']

In [58]:
tweet = vec._get_vector_list(tweet)
tweet = tweet.reshape(1,tweet.shape[0],tweet.shape[1])

In [59]:
tweet.shape

(1, 5, 300)

In [60]:
tweet = vec.padding_truncate_lists(tweet, max_length=20)

In [61]:
tweet.shape

(1, 20, 300)

In [63]:
np.argmax(model.predict(tweet))

1/1 [==============================] - 0s 38ms/step


2